<a href="https://colab.research.google.com/github/dirkenglund/quantum-network-code/blob/main/2023_08_Boston_fiber_network_graph_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CQN Quantum Network Links

author: Dirk Englund
latest rev: 20241020

In [ ]:
!pip install networkx

!pip install plotly

!pip install contextily
!pip install geopandas
!pip install gmaps
!pip install mplleaflet

Dirk's test 2023.10.23

Dirk revised 20241020

In [41]:
# Install necessary packages
#!pip install folium networkx

print("S")

import folium
import networkx as nx
from dataclasses import dataclass, field

# -------------------- DATA CLASSES --------------------

@dataclass
class Node:
    uid: str
    name: str
    position: tuple
    memory_type: str
    cooling: str
    T2: str
    status: str
    participants: str
    technologies: list = field(default_factory=list)

@dataclass
class Edge:
    start_uid: str
    end_uid: str
    label: str
    length: float  # in meters
    transmission: float  # in dB

# -------------------- NODE DEFINITIONS --------------------

# Define nodes using the Node data class
nodes = {
    "FH576-A": Node("FH576-A", "Fiber Hub 576-A", (42.3619, -71.0903), "N/A", "4K", "N/A", "Online", "CQN"),
    "HARV": Node("HARV", "Harvard", (42.3656, -71.1032), "SiV", "1K", ">1ms", "Installed", "MIT-LL", ["Quantum Frequency Conversion"]),
    "LINCOLN": Node("LINCOLN", "Lincoln Lab", (42.4464, -71.2258), "SiV", "0.3K", ">1ms", "Under Construction", "Startup", ["Quantum Frequency Conversion"]),
    "FH576-B": Node("FH576-B", "Fiber Hub 576-B", (42.3619, -71.0903), "N/A", "4K", "?", "Online", "CQN"),
    "26-465": Node("26-465", "26-465", (42.3646, -71.1028), "SiV", "1K", ">1ms", "Installed", "MIT-LL"),
    "38-177": Node("38-177", "38-177 SE Wall", (42.3593, -71.0957), "SnV", "0.3K", "?", "Online", "Startup"),
    "26-368": Node("26-368", "26-368", (42.3646, -71.1028), "SnV", "4K", "?", "Online", "CQN"),
    "36-576": Node("36-576", "36-576", (42.3619, -71.0903), "SnV, SiV, silicon G-center", "1K", "?", "Installed", "MIT-LL"),
    "NOTAROS": Node("NOTAROS", "Notaros Lab", (42.3646, -71.1028), "N/A", "0.3K", "N/A", "Online", "Startup"),
    "38-185": Node("38-185", "38-185", (42.3593, -71.0957), "SnV", "1K", ">2ms", "Online", "CQN"),
    "UA": Node("UA", "University of Arizona", (32.2319, -110.9501), "SnV", "4K", "N/A", "Online", "CQN"),
    "COQREATE": Node("COQREATE", "CoQreate (Ireland)", (53.349805, -6.26031), "SnV", "N/A", "50ms", "Installed", "MIT-LL"),
    "MITLL": Node("MITLL", "MIT Lincoln Lab", (42.4464, -71.2258), "SiV", "0.1K", ">1ms", "Under Construction", "Startup", ["Quantum Frequency Conversion"]),
    "QUNET": Node("QUNET", "QuNet", (42.3601, -71.0589), "SiV", "4K", "n/a", "Online", "CQN"),
    "UO": Node("UO", "University of Oregon", (44.0445, -123.0122), "N/A", "1K", "N/A", "Installed", "MIT-LL"),
    "BYU": Node("BYU", "BYU", (40.2338, -111.6584), "N/A", "0.3K", "N/A", "Online", "Startup"),
    "HU": Node("HU", "Howard University", (38.9175, -77.0189), "N/A", "4K", "N/A", "Online", "CQN"),
    "UMASS": Node("UMASS", "UMass Amherst", (42.3902, -72.5297), "N/A", "1K", "N/A", "Installed", "MIT-LL"),
    "UMD": Node("UMD", "University of Maryland", (38.9856, -76.9426), "trapped ions", "RT", ">1 s", "Online", "Startup"),
    "TRINITY": Node("TRINITY", "Trinity College (Ireland)", (53.3434, -6.2540), "SnV", "<1K", "?", "Online", "CQN"),
    "BU": Node("BU", "Boston University Photonics Incubator", (42.350876, -71.106918), "SiV", "<1K", "?", "Installed", "QuNETT"),
    "ATT_HUB": Node("ATT_HUB", "AT&T Fiber Hub", (42.3633196, -71.0925566), "N/A", "N/A", "N/A", "Online", "AT&T"),
    "ENG_LAB": Node("ENG_LAB", "Englund Lab (36-575)", (42.3615, -71.0930), "SiV, SnV", "4K", "N/A", "Online", "MIT"),
    "BBN": Node("BBN", "BBN Raytheon", (42.3897538, -71.1475561), "N/A", "<100mK", "N/A", "Online", "BBN"),
}

# -------------------- EDGE DEFINITIONS --------------------

# Define edges using the Edge data class
edges = [
    Edge("FH576-A", "HARV", "2 SMF-28", 100, -2),  # Fiber Hub 576-A to Harvard
    Edge("FH576-A", "LINCOLN", "2 SMF-28", 120, -2.5),  # Fiber Hub 576-A to Lincoln Lab
    Edge("FH576-B", "26-465", "4 ?,", 80, -3),        # Fiber Hub 576-B to 26-465
    Edge("FH576-B", "26-465", "2 SM600", 70, -3.5),    # Fiber Hub 576-B to 26-465
    Edge("FH576-B", "LINCOLN", "2 ?", 60, -2.8),       # Fiber Hub 576-B to Lincoln Lab
    Edge("26-368", "36-576", "4 SMF28", 90, -3),     # 26-368 to 36-576
    Edge("26-368", "36-576", "2 780HP", 95, -3),     # 26-368 to 36-576
    Edge("26-368", "36-576", "2 630HP", 85, -3),     # 26-368 to 36-576
    Edge("26-368", "36-576", "2 PM63-U40D", 75, -3), # 26-368 to 36-576
    Edge("26-368", "36-576", "4 SMF28 (Metal Sheethed)", 100, -3), # 26-368 to 36-576
    Edge("26-368", "36-576", "2 PM40-U40D", 80, -3), # 26-368 to 36-576
    Edge("26-368", "38-185", "6 SMF28", 70, -3),      # 26-368 to 38-185
    Edge("26-368", "26-465", "4 SMF28", 85, -3),      # 26-368 to 26-465
    Edge("26-368", "NOTAROS", "2 PM40-U40D", 95, -3),  # 26-368 to Notaros Lab
    Edge("26-368", "NOTAROS", "2 630HP", 85, -3),       # 26-368 to Notaros Lab
    Edge("26-368", "NOTAROS", "2 780HP", 95, -3),       # 26-368 to Notaros Lab
    Edge("26-368", "NOTAROS", "2 SMF-28", 90, -3),      # 26-368 to Notaros Lab
    Edge("FH576-A", "BU", "1 SMF-28", 110, -2),          # Fiber Hub 576-A to Boston University
    Edge("BU", "UMD", "2 SMF-28", 150, -1.5),             # Boston University to University of Maryland
    Edge("COQREATE", "TRINITY", "1 SMF-28", 200, -1.8),   # CoQreate (Ireland) to Trinity College (Ireland)
    Edge("UMASS", "UO", "2 SMF-28", 130, -1.7),           # UMass Amherst to University of Oregon
    Edge("HU", "UMD", "1 SMF-28", 140, -2),               # Howard University to University of Maryland
    Edge("ENG_LAB", "ATT_HUB", "2 SMF-28", 60, -1.2),     # Englund Lab to AT&T Fiber Hub
    Edge("ATT_HUB", "MITLL", "3 SMF-28", 80, -1.5),      # AT&T Fiber Hub to MIT Lincoln Lab
    Edge("ATT_HUB", "HARV", "3 SMF-28", 85, -1.5),        # AT&T Fiber Hub to Harvard
    Edge("ATT_HUB", "BBN", "3 SMF-28", 90, -1.5),         # AT&T Fiber Hub to BBN Raytheon
]

# -------------------- BUILD NETWORK GRAPH --------------------

# Create a geographic graph
G = nx.Graph()

# Add all nodes to the graph with positions
for node, attributes in nodes.items():
    G.add_node(attributes.uid, pos=attributes.position)

# Add all edges to the graph
for edge in edges:
    G.add_edge(edge.start_uid, edge.end_uid, label=edge.label, length=edge.length, transmission=edge.transmission)

# -------------------- CREATE FOLIUM MAP --------------------

# Create a Folium map centered around Boston
m = folium.Map(location=[42.3601, -71.0589], zoom_start=5, tiles='cartodb positron')

# Add nodes to the map with detailed labels
for node, attributes in nodes.items():
    lat, lon = attributes.position
    info = (
        f"{attributes.name}<br>"
        f"Memory type: {attributes.memory_type}<br>"
        f"Cooling: {attributes.cooling}<br>"
        f"T2: {attributes.T2}<br>"
        f"Status: {attributes.status}<br>"
        f"Participants: {attributes.participants}<br>"
        f"Technologies: {', '.join(attributes.technologies)}"
    )
    folium.Marker(location=[lat, lon], popup=info, icon=folium.Icon(color='blue')).add_to(m)

# Add edges to the map with curved lines
def add_curved_edge(start, end, label, color='blue', dash_array=None):
    start_node = G.nodes[start]
    end_node = G.nodes[end]

    # Create a curved line effect by adding intermediary points
    mid_point = [(start_node['pos'][0] + end_node['pos'][0]) / 2, (start_node['pos'][1] + end_node['pos'][1]) / 2 + 0.01]  # Adjust for curvature
    locations = [start_node['pos'], mid_point, end_node['pos']]

    folium.PolyLine(locations=locations, color=color, weight=2.5, dash_array=dash_array, popup=label).add_to(m)

# Add existing edges (solid blue lines)
for edge in G.edges(data=True):
    add_curved_edge(edge[0], edge[1], edge[2]['label'], color='blue')

# Add planned links (dashed red lines)
add_curved_edge("FH576-A", "UA", "Planned Link: Joint Device Development", color='red', dash_array='5, 5')

# -------------------- ADD LEGEND --------------------

# Create a legend for the map
legend_html = """
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 200px; height: auto;
     background-color: white; border:2px solid grey; z-index:9999;
     font-size:14px; padding: 10px;">
&emsp;<b>Legend</b><br>
&emsp;<i style="color:blue;">&#9679;</i>&nbsp;Existing Edge<br>
&emsp;<i style="color:red;">&#9679;</i>&nbsp;Planned Link<br>
&emsp;<i style="color:gray;">&#9679;</i>&nbsp;Fiber Connection<br>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# Save the map
m.save("boston_network_map.html")
m

S
